In [3]:
import numpy as np
import pandas as pd

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [41]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [6]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [7]:
X = pd.read_csv('./pts.csv', index_col=0)
X = X.drop(['system_id', 'status_id', 'time', 'matrix_id'], axis=1)

In [8]:
X_vals = X.values
X_vals.shape

(871535, 79)

In [47]:
y = pd.read_csv('./labels.csv', index_col=0).as_matrix()
lbl_error = y[:,1]
lbl_error

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [53]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(79, input_dim=79, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [54]:
# evaluate model with standardized dataset
with tf.device('/gpu:0'):
    estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=8192)
    kfold = StratifiedShuffleSplit(n_splits=10, random_state=seed)
    results = cross_val_score(estimator, X_vals, lbl_error, cv=kfold)
    print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

    #Original Results: 49.28% (8.12%)
    #Standardized Results: 60.06% (0.11%)

Epoch 1/10
784381/784381 [==============================] - 2s - loss: 3.3643 - acc: 0.7912     
Epoch 2/10
784381/784381 [==============================] - 1s - loss: 3.3035 - acc: 0.7950     
Epoch 3/10
784381/784381 [==============================] - 1s - loss: 3.3035 - acc: 0.7950     
Epoch 4/10
784381/784381 [==============================] - 1s - loss: 3.3035 - acc: 0.7950     
Epoch 5/10
784381/784381 [==============================] - 1s - loss: 3.3035 - acc: 0.7950     
Epoch 6/10
598016/784381 [=====================>........] - ETA: 0s - loss: 3.3062 - acc: 0.7949

KeyboardInterrupt: 

In [56]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=10, batch_size=8192)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, lbl_error, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/10
784381/784381 [==============================] - 3s - loss: 0.5261 - acc: 0.7843     
Epoch 2/10
784381/784381 [==============================] - 2s - loss: 0.4946 - acc: 0.7950     
Epoch 3/10
784381/784381 [==============================] - 2s - loss: 0.4928 - acc: 0.7950     
Epoch 4/10
784381/784381 [==============================] - 2s - loss: 0.4912 - acc: 0.7950     
Epoch 5/10
784381/784381 [==============================] - 2s - loss: 0.4895 - acc: 0.7951     
Epoch 6/10
784381/784381 [==============================] - 2s - loss: 0.4879 - acc: 0.7952     
Epoch 7/10
784381/784381 [==============================] - 2s - loss: 0.4864 - acc: 0.7955     
Epoch 8/10
784381/784381 [==============================] - 2s - loss: 0.4849 - acc: 0.7959     
Epoch 9/10
784381/784381 [==============================] - 2s - loss: 0.4834 - acc: 0.7962     
Epoch 10/10
87154/87154 [==============================] - 1s     
Epoch 1/10
784381/784381 [==============================] - 

87153/87153 [==============================] - 1s     
Epoch 1/10
784382/784382 [==============================] - 4s - loss: 0.5305 - acc: 0.7746     
Epoch 2/10
784382/784382 [==============================] - 2s - loss: 0.4946 - acc: 0.7950     
Epoch 3/10
784382/784382 [==============================] - 2s - loss: 0.4932 - acc: 0.7951     
Epoch 4/10
784382/784382 [==============================] - 2s - loss: 0.4918 - acc: 0.7951     
Epoch 5/10
784382/784382 [==============================] - 2s - loss: 0.4900 - acc: 0.7952     
Epoch 6/10
784382/784382 [==============================] - 2s - loss: 0.4882 - acc: 0.7952     
Epoch 7/10
784382/784382 [==============================] - 2s - loss: 0.4865 - acc: 0.7954     
Epoch 8/10
784382/784382 [==============================] - 2s - loss: 0.4849 - acc: 0.7959     
Epoch 9/10
784382/784382 [==============================] - 2s - loss: 0.4836 - acc: 0.7963     
Epoch 10/10
87153/87153 [==============================] - 1s     
Epoch